In [11]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.pipeline import Pipeline

In [39]:
# Train Ddata import
df = pd.read_csv('amazon_all_train.csv', sep = '|')
df.dropna(inplace=True)
len(df)

1343648

In [40]:
#df = df.sample(1000000, random_state=42)
#len(df)

In [41]:
%%time
# Split dataset
# X_train, X_test, y_train, y_test = train_test_split(df.text, df.label, test_size=0.1, random_state=42, stratify=df.label)
X_train, y_train = (df.text, df.label)

#indexes = np.arange(len(X_train))
#np.random.seed(42)
#np.random.shuffle(indexes)
#X_train, y_train = X_train[indexes], y_train[indexes]
X_train = X_train.astype(str)
#X_train.dropna(inplace=True)

CPU times: user 440 ms, sys: 37.3 ms, total: 478 ms
Wall time: 509 ms


In [42]:
# Take the last 22 words from each review in the train set
#X_train = X_train.str.split().apply(lambda x:  ' '.join(x for x in x[-22:]))

In [43]:
# Stopwords
STOPWORDS = ['by','does', 'was', 'were', 'the', 'of', 'end', 'and', 'is']    

In [44]:
cvect = CountVectorizer()
classifier = PassiveAggressiveClassifier(C=0.001, fit_intercept = False, shuffle = False, n_iter = 91)

pipeline = Pipeline([('vectorizer', CountVectorizer(binary=True,ngram_range=(1,4),stop_words=STOPWORDS)), ('classifier', classifier)])

In [45]:
%%time

model = pipeline.fit(X_train, y_train)

CPU times: user 14min 24s, sys: 17min 51s, total: 32min 16s
Wall time: 43min 4s


In [46]:
%%time

amz = pd.read_csv('amazon_all_test.csv', sep = '|')
rt = pd.read_csv('rt_test.csv', sep = '|')

CPU times: user 2.24 s, sys: 999 ms, total: 3.24 s
Wall time: 6.54 s


In [47]:
X_test_amz, y_test_amz = (amz.text, amz.label)
X_test_rt, y_test_rt = (rt.text, rt.label)

In [48]:
%%time

y_pred_amz = model.predict(X_test_amz.values.astype('U'))
y_pred_rt = model.predict(X_test_rt)

CPU times: user 4min 35s, sys: 7min 12s, total: 11min 48s
Wall time: 19min 33s


In [49]:
print ("Accuracy Amazon :", metrics.accuracy_score(y_test_amz, y_pred_amz))
print ("F1 Amazon :", metrics.f1_score(y_test_amz, y_pred_amz, pos_label=0))

print ("Accuracy RT :", metrics.accuracy_score(y_test_rt, y_pred_rt))
print ("F1 RT :", metrics.f1_score(y_test_rt, y_pred_rt, pos_label=0))

Accuracy Amazon : 0.929805760214
F1 Amazon : 0.674493725929
Accuracy RT : 0.702251407129
F1 RT : 0.601456554495
